In [1]:
import numpy as np
import pandas as pd
import glob
import re
import os
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

import pylab as plt
import seaborn as sns
from statannot import add_stat_annotation
from functools import reduce

import itertools
import sys
sys.path.insert(1,'/scratch/c.c21013066/software/biobankAccelerometerAnalysis/accelerometer')
sys.path.insert(1,'/scratch/c.c21013066/UKBIOBANK_DataPreparation/phenotypes')
import utils
import plots

In [2]:
img_path = '/scratch/c.c21013066/images/ukbiobank/accelerometer/features'
data_path = '/scratch/c.c21013066/data/ukbiobank'
sample_path = '/scratch/c.c21013066/data/ukbiobank/sample/withGP/noOsteo'
def plot_context():
    sns.set_context("talk", rc={"font.size":18,"axes.titlesize":18,"axes.labelsize":16,"font_scale":0.9})

# Prepare bulk file which to download

In [4]:
dfs = []
for name in ['Osteoarthritis']:#['AllCauseDementia','AllCauseParkinsonism','AlzheimerDisease','ParkinsonDisease',
        #'MultipleSystemAtrophy','ProgressiveSupranuclearPalsy','FrontoTemporalDementia','VascularDementia',
        #    'Dystonia']:#,'Osteoarthritis']:#,'Depression']:
        drop_healthy='neurology'
        merged_clean = pd.read_csv(f'{sample_path}/{name}_controlNo{drop_healthy}_acc_QC.csv').set_index('eid')
        xnames = merged_clean.filter(regex='Average').columns.str.replace('Average','').str.replace('___','-')
        diag_name = name
        merged_clean['diagnosis'] = merged_clean[name].replace([0,1],['Healthy',diag_name])
        merged_clean['acc_time_since_diagnosis'] = merged_clean[f'accelerometry_age'] - merged_clean[f'{name}_age']
        merged_clean['acc_time_to_diagnosis'] =  merged_clean[f'{name}_age'] - merged_clean[f'accelerometry_age']
        merged_clean['acc_incident'] = merged_clean[f'{name}_age'] > merged_clean[f'accelerometry_age']
        merged_clean.loc[merged_clean['acc_time_since_diagnosis'].isna(),'acc_incident'] = np.nan
        merged_clean['diagnosis_prod'] = merged_clean['diagnosis'].copy(deep=True)
        merged_clean.loc[np.logical_and(merged_clean['acc_incident']==1,merged_clean['diagnosis']==diag_name),
                         'diagnosis_prod'] = f'Prodromal_{diag_name}'
        merged_clean['diagnosis_prod_conservative'] = merged_clean['diagnosis_prod'].copy(deep=True)
        merged_clean.loc[np.logical_and(merged_clean['acc_time_to_diagnosis']<2,
                                        merged_clean['diagnosis_prod']==f'Prodromal_{diag_name}'),
                         'diagnosis_prod_conservative'] = diag_name
        merged_clean = merged_clean[merged_clean['weartime_QC']==1]
        matched_sample = merged_clean[merged_clean['diagnosis']==diag_name]
        dfs.append(matched_sample)
        
df_merged = pd.concat(dfs)

In [ ]:
# drop duplicates
print(df_merged.shape)
df_merged = df_merged[~df_merged.index.duplicated(keep='first')]
print(df_merged.shape)

In [63]:
# get all downloadable IDs
acc_ids = pd.read_csv(f'{data_path}/ukb50687.bulk',usecols=[0,1],header=None,
                      delim_whitespace=True,names=['eid','field_id']).set_index('eid')
drop_seasonal_repeats = acc_ids[~acc_ids.index.duplicated(keep='first')]
n_visits = (acc_ids.groupby(level=0).size()).rename('n_visits')
acc_ids = pd.merge(acc_ids,n_visits,right_index=True,left_index=True,how='outer')

In [ ]:
get_acc = pd.merge(drop_seasonal_repeats,df_merged.reset_index(),on='eid',how='right').set_index('eid')

In [6]:
get_acc.to_csv(f'{data_path}/ukb50687_90001_0_0_Osteoarthritis.bulk',header=False,sep=' ')
get_acc.to_csv(f'/scratch/scw1329/annkathrin/data/ukbiobank/ukb50687_90001_0_0_Osteoarthritis.bulk',header=False,sep=' ')

In [8]:
get_acc = pd.read_csv(f'{data_path}/ukb50687_90001_0_0_Osteoarthritis.bulk',sep=' ',
                      names=['eid','field'],usecols=[0,1]).set_index('eid')
# drop already downloaded ones
paths = [f'{data_path}/phenotypes','/scratch/scw1329/annkathrin/data/ukbiobank']
eids = []
for path in paths:
    folders = glob.glob(f"{path}/accelerometer/*", recursive = True)
    #print(folders)
    downloaded = []
    downloaded.append(glob.glob(f"{path}/accelerometer/*.cwa"))
    downloaded.append(glob.glob(f"{path}/accelerometer/*timeSeries.csv.gz"))
    downloaded = list(itertools.chain(*downloaded))
    eids.append([int(re.findall(r'\d+', headline)[1]) for headline in downloaded])
    downloaded = []
    for folder in folders:
        downloaded.append(glob.glob(f"{folder}/*.cwa"))
        downloaded.append(glob.glob(f"{folder}/*timeSeries.csv.gz"))
# get eid
eids = list(itertools.chain(*eids))
downloaded = list(itertools.chain(*downloaded))
eids_add = [int(re.findall(r'\d+', headline)[2]) for headline in downloaded]
eids = np.unique(np.hstack([eids,eids_add]))
eids_intersect = np.intersect1d(get_acc.index,eids)

In [9]:
get_acc['downloaded'] = False
get_acc.loc[eids_intersect,'downloaded'] = True
get_acc = get_acc[get_acc['downloaded']==False].drop(columns='downloaded')
print(get_acc.shape)

(6633, 1)


In [10]:
get_acc.to_csv(f'/scratch/scw1329/annkathrin/data/ukbiobank/ukb50687_90001_0_0_Osteoarthritis_remains.bulk',header=False,sep=' ')
get_acc.to_csv(f'{data_path}/ukb50687_90001_0_0_Osteoarthritis_remains.bulk',header=False,sep=' ')

In [246]:
# Storgae problems, so delete .cwa files are zipping them and storing on other disc
# which ones to delete
folders = glob.glob(f"{data_path}/phenotypes/accelerometer/*/", recursive = True)
print(folders)
alltimes = []
allraw = []
for folder in [f'{data_path}/phenotypes/accelerometer/cwa_depr_3/']:
    alltimes.append(glob.glob(f"{folder}/*timeSeries.csv.gz"))
    allraw.append(glob.glob(f"{folder}/*.cwa"))
alltimes = list(itertools.chain(*alltimes))
allraw = list(itertools.chain(*allraw))
print(len(alltimes),len(allraw))
alltimeseid = [int(re.findall(r'\d+', headline)[2]) for headline in alltimes]
allraweid = [int(re.findall(r'\d+', headline)[2]) for headline in allraw]
delete = np.setdiff1d(alltimeseid,allraweid)
delete = np.intersect1d(allraweid,alltimeseid)
len(delete),len(alltimeseid)

['/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_10001/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_3/']
0 397


(0, 0)

In [ ]:
for f in delete:
    os.remove(os.path.join('/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_3/',f'{f}_90001_0_0.cwa'))

# Create file to lookup subjects

In [80]:
# create lookup list of subject_eid folder
paths = [f'{data_path}/phenotypes','/scratch/scw1329/annkathrin/data/ukbiobank']
alltimes = []
for path in paths:
    alltimes.append(glob.glob(f"{path}/accelerometer/*timeSeries.csv.gz"))
    folders = glob.glob(f"{path}/accelerometer/*/", recursive = True)
    for folder in folders:
        alltimes.append(glob.glob(f"{folder}/*timeSeries.csv.gz"))
alltimes = list(itertools.chain(*alltimes))
alltimes = np.unique(alltimes)

In [85]:
table = pd.DataFrame(columns=['path','folder','eid','file'])
table['file'] = alltimes
table['folder'] = table['file'].str.split("/").str[-2]#table['file'].str.findall(r'(\d+)/\d+_90001_0_0').str[0]
table['path'] = table['file'].str.split("/").str[:-1].apply('/'.join)
table['eid'] = table['file'].str.findall(r'(\d+)_90001_0_0').str[-1].astype(int)

In [86]:
print(len(np.unique(table['eid'])),table.shape)
table = table.drop_duplicates(subset=['eid'],keep='first')

65902 (65928, 4)


In [87]:
table.to_csv('/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/subject_file_lookup.csv')

# Merge processed files

In [88]:
# create lookup list of subject_eid folder
paths = [f'{data_path}/phenotypes','/scratch/scw1329/annkathrin/data/ukbiobank']
alltimes = []
for path in paths:
    alltimes.append(glob.glob(f"{path}/accelerometer/*allsubject*_summary_from_raw.csv"))
    folders = glob.glob(f"{path}/accelerometer/*/", recursive = True)
    print(folders)
    for folder in folders:
        alltimes.append(glob.glob(f"{folder}/*allsubject*_summary_from_raw.csv"))
alltimes = list(itertools.chain(*alltimes))
alltimes = np.unique(alltimes)

['/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/to_process2/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_10001/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_3/']
['/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_17002/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_31802/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_32102/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_Ost_971/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_34402/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_27402/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_1902/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_32302/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_Ost_6571/', '/scratch/scw1329/annkathrin/data/ukbiobank/accelerometer/cwa_HC_26702/', '/scratch/scw1329/annkathrin/dat

In [89]:
files = []
for file in alltimes:
    files.append(pd.read_csv(file))
#merged = reduce(lambda  left,right: pd.merge(left,right,on=['eid'],
#                                            how='outer'), files)
merged = pd.concat(files)

In [90]:
print(merged.shape)
merged = merged.dropna(subset=['covered_days'],how='all',axis='rows')
print(merged.shape)
unique = merged.drop_duplicates(subset=['eid'],keep='first')

(65902, 80)
(65902, 80)


In [91]:
# merge with known accelerometer info and expand to dataset
acc = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/phenotypes/demo_acc.csv',index_col=0)
acc = acc[acc.columns[34:72]]

In [92]:
merged_acc = pd.merge(acc.reset_index(),unique,on=['eid'],how='inner')

In [93]:
merged_acc.filter(regex='complete').describe()

,complete_days_starting_10h,complete_days_starting_0h,complete_days_starting_7h
count,65901.000000,65901.000000,50.0
mean,6.947269,5.958544,6.0
std,0.365711,0.315477,0.0
min,2.000000,2.000000,6.0
25%,7.000000,6.000000,6.0
50%,7.000000,6.000000,6.0
75%,7.000000,6.000000,6.0
max,7.000000,6.000000,6.0


In [94]:
merged_acc.to_csv('/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/alluniquesubjects_summary_fromraw_fulldays.csv')

## Get matched data file

In [46]:
dfs = pd.read_csv(f'{data_path}/phenotypes/accelerometer/unmatched_all_HCnoOsteo_genebloodrisk.csv',index_col=0)

In [47]:
PDids2 = dfs[dfs['diagnosis']=='ParkinsonDisease'].index

In [43]:
blood = pd.read_csv(f'{sample_path}/ParkinsonDisease_controlNononHC_blood.csv',index_col=0)

In [48]:
blood.loc[PDids2]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([1008826, 1038369, 1047057, 1051759, 1092128,\n            ...\n            5909394, 5947793, 5952015, 5972045, 5996143],\n           dtype='int64', name='eid', length=152). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [21]:
acc = pd.read_csv(f'{data_path}/phenotypes/accelerometer/alluniquesubjects_summary_fromraw_fulldays.csv',
                 index_col=0).set_index('eid')

In [40]:
risks = []
for disease in ['AllCauseDementia','AllCauseParkinsonism','AlzheimerDisease','ParkinsonDisease','Dystonia','Osteoarthritis','Depression']:
    risk = pd.read_csv(f'{sample_path}/{disease}_controlNononHC_risk.csv',index_col=0)
    blood = pd.read_csv(f'{sample_path}/{disease}_controlNononHC_blood.csv',index_col=0)
    riskblood = pd.merge(risk,blood,right_index=True,left_index=True,how='outer',suffixes=['_drop',''])
    riskblood = riskblood.drop(columns=riskblood.filter(regex='_drop').columns)
    risks.append(riskblood)
riskblood = pd.concat(risks)
riskblood = riskblood[~riskblood.index.duplicated(keep='first')]

In [16]:
merged = pd.merge(acc,dfs,right_index=True,left_index=True,how='right',suffixes=['_drop',''])
merged = merged.drop(columns=merged.filter(regex='_drop').columns)

In [102]:
m.groupby('diagnosis')[['covered_days','complete_days_starting_10h','complete_days_starting_0h']].agg(['count','mean','std','min','max'])

covered_days                                          \
                            count      mean       std       min       max   
diagnosis                                                                   
AllCauseDementia             1369  6.973786  0.233289  3.127083  7.000694   
AllCauseParkinsonism          519  6.964806  0.298386  3.215625  7.001042   
AlzheimerDisease              212  6.983664  0.127796  5.494792  7.000347   
Depression                   9586  6.963362  0.276753  3.084028  7.001736   
Dystonia                       81  6.970383  0.260301  4.656597  6.999306   
Healthy                     43753  6.960136  0.300381  2.941319  7.001736   
Osteoarthritis              13407  6.957053  0.307795  2.994792  7.002083   
ParkinsonDisease              469  6.961128  0.313697  3.215625  7.001042   

                     complete_days_starting_10h                              \
                                          count      mean       std min max   
diagnosis                                                                     
AllCauseDementia                           1369  6.965668  0.285347   3   7   
AllCauseParkinsonism                        519  6.953757  0.359349   3   7   
AlzheimerDisease                            212  6.976415  0.180600   5   7   
Depression                                 9586  6.950031  0.344427   3   7   
Dystonia                                     81  6.962963  0.333333   4   7   
Healthy                                   43753  6.947958  0.365552   2   7   
Osteoarthritis                            13407  6.944283  0.374419   2   7   
ParkinsonDisease                            469  6.948827  0.377723   3   7   

                     complete_days_starting_0h                              
                                         count      mean       std min max  
diagnosis                                                                   
AllCauseDementia                          1369  5.972973  0.247797   2   6  
AllCauseParkinsonism                       519  5.965318  0.302422   2   6  
AlzheimerDisease                           212  5.981132  0.167566   4   6  
Depression                                9586  5.962863  0.288319   2   6  
Dystonia                                    81  5.975309  0.222222   4   6  
Healthy                                  43753  5.958974  0.315944   2   6  
Osteoarthritis                           13407  5.955396  0.326296   2   6  
ParkinsonDisease                           469  5.961620  0.317943   2   6

In [17]:
m = pd.merge(merged,riskblood[riskblood.columns[:-18]],right_index=True,left_index=True,how='left',suffixes=['_drop',''])
m = m.drop(columns=m.filter(regex='_drop').columns)
m.to_csv(f'{data_path}/phenotypes/accelerometer/unmatched_all_HCnoOsteo_riskblood.csv')

In [13]:
subset = m[m['diagnosis']=='ParkinsonDisease']
hc = subset[subset['Status']=='Healthy']
hc.shape

(469, 236)

In [14]:
hc.isna().sum()

Unnamed: 0                      0
covered_days                    0
complete_days_starting_10h      0
complete_days_starting_0h       0
complete_days_starting_7h     469
                             ... 
Testosterone                    0
Totalprotein                    0
Triglycerides                   0
Urate                           0
VitaminD                        0
Length: 236, dtype: int64

In [120]:
merged.to_csv(f'{data_path}/phenotypes/accelerometer/matched_all_HCnoOsteo.csv')

In [47]:
dfs = pd.read_csv(f'{data_path}/phenotypes/accelerometer/unmatched_all_HCnoOsteo_NoDepressedNoPD.csv',index_col=0)
print(dfs.columns[100:200])
dfs = pd.read_csv(f'{data_path}/phenotypes/accelerometer/matched_all_HCnoOsteo_NoDepressedNoPD.csv',index_col=0)
print(dfs.columns[100:200])

Index(['male', 'TownsendDeprivationIndex', 'AlcoholStatus_Current',
       'AlcoholStatus_Previous', 'SmokeStatus_Current', 'SmokeStatus_Previous',
       'DaytimeSleepiness_Often', 'AlcoholFrequency_LessThanWeekly',
       'family_Stroke', 'family_Diabetes', 'family_Severedepression',
       'family_Alzheimersdiseasedementia', 'family_Parkinsonsdisease', 'BMI',
       'Waist_Circumference', 'Hip_Circumference', 'Diastolic_BloodPressure',
       'PulseRate', 'BodyFat_Percentage', 'Albumin', 'Alkalinephosphatase',
       'Alanineaminotransferase', 'ApolipoproteinA', 'ApolipoproteinB',
       'Aspartateaminotransferase', 'Urea', 'Calcium', 'Cholesterol',
       'Creatinine', 'C_reactiveprotein', 'CystatinC',
       'Gammaglutamyltransferase', 'Glucose', 'Glycatedhaemoglobin_HbA1c',
       'HDLcholesterol', 'IGF_1', 'LDLdirect', 'Phosphate', 'SHBG',
       'Totalbilirubin', 'Testosterone', 'Totalprotein', 'Triglycerides',
       'Urate', 'VitaminD', 'AllCauseDementia_age', 'time_to_diagno

In [104]:
# HC population
acc = pd.read_csv(f'{data_path}/phenotypes/accelerometer/alluniquesubjects_summary_fromraw_fulldays.csv').set_index('eid')
hc = pd.read_csv(f'{data_path}/phenotypes/accelerometer/allHCnoOsteo.csv',index_col=0)
hc = pd.merge(hc,acc,right_index=True,left_index=True,how='left',suffixes=['','_drop'])
hc = hc.drop(columns=hc.filter(regex='_drop').columns)
hc.to_csv(f'{data_path}/phenotypes/accelerometer/allHCnoOsteo.csv')

## Check if all included

In [70]:
# time-series files
downloaded = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/subject_file_lookup.csv').set_index('eid')
extracted = pd.read_csv('/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/alluniquesubjects_summary_fromraw_fulldays.csv',
                 index_col=0).set_index('eid')
diff = np.setdiff1d(downloaded.index,extracted.index)
downloaded.loc[diff]

,Unnamed: 0,path,folder,file
eid,,,,
2667457,7905,/scratch/c.c21013066/data/ukbiobank/phenotypes...,accelerometer,/scratch/c.c21013066/data/ukbiobank/phenotypes...


In [44]:
downloaded.loc[diff].to_csv('/scratch/scw1329/annkathrin/data/ukbiobank/to_process4.csv')

# Remove processed files

In [10]:
# which ones to delete
folders = glob.glob("/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/*/", recursive = True)
print(folders)
alltimes = []
allraw = []
for folder in ['/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_6002/']:
    alltimes.append(glob.glob(f"{folder}/*-summary.json"))
    allraw.append(glob.glob(f"{folder}/*.cwa"))
alltimes = list(itertools.chain(*alltimes))
allraw = list(itertools.chain(*allraw))
print(len(alltimes),len(allraw))
alltimeseid = [int(re.findall(r'\d+', headline)[2]) for headline in alltimes]
allraweid = [int(re.findall(r'\d+', headline)[2]) for headline in allraw]
delete = np.setdiff1d(alltimeseid,allraweid)
delete = np.intersect1d(allraweid,alltimeseid)
len(delete),len(alltimeseid)

['/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_10001/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_10001/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_8001/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_6002/', '/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_3/']
247 0


(0, 247)

In [11]:
for f in delete:
    os.remove(os.path.join('/scratch/c.c21013066/data/ukbiobank/phenotypes/accelerometer/cwa_depr_6002/',f'{f}_90001_0_0.cwa'))